## Library Installation

Ensure that the necessary libraries are installed before running the notebook.

In [1]:
# Import necessary libraries
%pip install openpyxl
from openpyxl import load_workbook
import pandas as pd
import os
import re
import itertools

print("Skeleton setup complete!")


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Skeleton setup complete!


## Variable Declaration

Set the variables for file paths, sheet names, and other configurations. Update these variables for each specific project.

In [2]:
# Path to the Excel file (change this for each project)
excel_file_path = '/workspaces/Finetwork-Automation/inbound/Metabase.xlsx'
csv_file_path = '/workspaces/Finetwork-Automation/inbound/Informe de métricas históricas.csv'

# Sheet names for different categories
sheet_active = 'ACTIVOS'
sheet_canceled = 'CANCELADOS'
sheet_pending = 'PTE DE FIRMA'

# Range to read (change this for each project)
start_row = 8
end_row = 65
usecols = 'A:AF'

print("Variables defined correctly!")

Variables defined correctly!


## Extract Data from "Informe de métricas históricas" CSV file
Extract data from the CSV file and convert it directly to a DataFrame.

In [3]:
import pandas as pd

# Asumiendo que ya has cargado y pivotado el DataFrame
df = pd.read_csv(csv_file_path)

pivot_df = df.pivot_table(index=['Agent', 'StartInterval', 'EndInterval'], 
                          values=['Contacts handled incoming', 'Contacts transferred out internal'], 
                          aggfunc='first')

# Verificar los niveles del índice
print("Niveles del índice:", pivot_df.index.names)

# Verificar las columnas del DataFrame
print("Columnas del DataFrame:", pivot_df.columns)

# Mostrar las primeras filas del DataFrame para verificar el contenido
print("Contenido del DataFrame:")
print(pivot_df.head())

# Verificar el tipo de datos del DataFrame
print("Tipo de datos:", type(pivot_df))

# Verificar los valores únicos en cada nivel del índice
print("Agentes únicos:", pivot_df.index.get_level_values('Agent').unique())
print("StartInterval únicos:", pivot_df.index.get_level_values('StartInterval').unique())
print("EndInterval únicos:", pivot_df.index.get_level_values('EndInterval').unique())



Niveles del índice: ['Agent', 'StartInterval', 'EndInterval']
Columnas del DataFrame: Index(['Contacts handled incoming', 'Contacts transferred out internal'], dtype='object')
Contenido del DataFrame:
                                                                                                Contacts handled incoming  \
Agent                              StartInterval                 EndInterval                                                
Irati.izaguirre@originaltelecom.es 2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       19.0   
                                   2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       23.0   
                                   2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       38.0   
                                   2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       43.0   
                                   2024-08-07T00:

## Load Agents List

Load the list of all agents from the "Agents" sheet.

In [4]:
# Load the list of agents
agents_df = pd.read_excel(excel_file_path, sheet_name='AGENTES', usecols='A')
agents_list = agents_df.iloc[:, 0].tolist()
print("Agents list loaded successfully!")
print(agents_list)

Agents list loaded successfully!
['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'anasanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'azahara.garcia@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'carmen.cornejo@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'formacion10@originaltelecom.es', 'formacion3@originaltelecom.es', 'formacion4@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'irati.izaguirre@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'lailasetati@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'mar.marchena@originaltelecom.es', 

## Verify and Complete Pivot Table Data

Verify that all agents are present in the Pivot Table. If an agent is missing, add a row with zeros for that agent.

In [5]:
def remove_extra_agents_from_pivot(pivot_df, agents_list):
    """
    Remove agents from the pivot DataFrame that are not in the agents_list.
    
    Parameters:
    pivot_df (pd.DataFrame): The pivoted DataFrame to update.
    agents_list (list): The list of all agents to keep.
    
    Returns:
    pd.DataFrame: The updated pivot DataFrame with only the agents in agents_list.
    """
    # Verificar si 'Agent' es un nivel de índice
    if 'Agent' not in pivot_df.index.names:
        pivot_df.index.names = ['Agent', 'StartInterval', 'EndInterval']
    
    # Filtrar el índice para mantener solo los agentes que están en agents_list
    mask = pivot_df.index.get_level_values('Agent').isin(agents_list)
    pivot_df = pivot_df[mask]
    
    # Verificar si se han eliminado los agentes no deseados
    remaining_unwanted_agents = [agent for agent in pivot_df.index.get_level_values('Agent').unique() if "formacion" in agent]
    if remaining_unwanted_agents:
        print(f"Los siguientes agentes no deseados siguen presentes: {remaining_unwanted_agents}")
    else:
        print("Todos los agentes no deseados han sido eliminados correctamente.")
    
    return pivot_df

# Asegúrate de que 'agents_list' no contiene agentes de "formación"
agents_list = [agent for agent in agents_list if "formacion" not in agent]

# Aplicar la función a la tabla pivoteada
pivot_df = remove_extra_agents_from_pivot(pivot_df, agents_list)

# Mostrar los agentes únicos después de filtrar
unique_agents = pivot_df.index.get_level_values('Agent').unique()
print("Agentes únicos después de filtrar:", unique_agents.tolist())


def add_missing_agents(pivot_df, agents_list):
    """
    Add missing agents to the pivot DataFrame with zero values for relevant columns.
    
    Parameters:
    pivot_df (pd.DataFrame): The pivoted DataFrame to update.
    agents_list (list): The list of all agents to ensure are present.
    
    Returns:
    pd.DataFrame: The updated pivot DataFrame with all agents.
    """
    # Verificar si 'Agent' es un nivel de índice, si no, restaurarlo
    if 'Agent' not in pivot_df.index.names:
        pivot_df.index.names = ['Agent', 'StartInterval', 'EndInterval']

    # Obtener la lista de agentes presentes en el DataFrame pivoteado
    existing_agents = pivot_df.index.get_level_values('Agent').unique()

    # Identificar agentes faltantes que están en la lista pero no en el pivot_df
    missing_agents = [agent for agent in agents_list if agent not in existing_agents]

    # Crear combinaciones de 'StartInterval' y 'EndInterval'
    date_combinations = list(itertools.product(
        pivot_df.index.get_level_values('StartInterval').unique(),
        pivot_df.index.get_level_values('EndInterval').unique()
    ))

    # Añadir filas para los agentes faltantes con valores cero
    for agent in missing_agents:
        for start_interval, end_interval in date_combinations:
            zero_row = pd.Series(
                [0, 0],  # Valores para 'Contacts handled incoming' y 'Contacts transferred out internal'
                index=pivot_df.columns,
                name=(agent, start_interval, end_interval)
            )
            pivot_df = pd.concat([pivot_df, zero_row.to_frame().T])

    print(f"Added {len(missing_agents)} missing agents.")
    return pivot_df

# Aplicar las funciones a la tabla pivoteada
informe_df = remove_extra_agents_from_pivot(pivot_df, agents_list)
informe_df = add_missing_agents(informe_df, agents_list)

# Mostrar el DataFrame actualizado
print("Informe DataFrame after ensuring all agents:")
display(informe_df.head(55))

# Verificar los niveles del índice
print("Niveles del índice:", pivot_df.index.names)

# Verificar las columnas del DataFrame
print("Columnas del DataFrame:", pivot_df.columns)

# Mostrar las primeras filas del DataFrame para verificar el contenido
print("Contenido del DataFrame:")
print(pivot_df.head())

# Verificar el tipo de datos del DataFrame
print("Tipo de datos:", type(pivot_df))

# Verificar los valores únicos en cada nivel del índice
# Mostrar todos los agentes únicos en el DataFrame
unique_agents = pivot_df.index.get_level_values('Agent').unique()

# Mostrar todos los agentes únicos
print("Agentes únicos:", unique_agents.tolist())
print("StartInterval únicos:", pivot_df.index.get_level_values('StartInterval').unique())
print("EndInterval únicos:", pivot_df.index.get_level_values('EndInterval').unique())



Todos los agentes no deseados han sido eliminados correctamente.
Agentes únicos después de filtrar: ['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'anasanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'mar.marchena@originaltelecom.es', 'maria.torres@originaltelecom.es', 'mariaarroyo@originaltelecom.es', 'mariangeles.bueso@originaltelecom.es', 'marta.dorado@originaltelecom.es', 'miriam.rodriguez@originaltelecom.e

Contacts handled incoming  \
albaaraujo@originaltelecom.es      2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       33.0   
                                   2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       29.0   
                                   2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       63.0   
                                   2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       46.0   
                                   2024-08-07T00:00:00.000+02:00 2024-08-08T00:00:00.000+02:00                       58.0   
                                   2024-08-08T00:00:00.000+02:00 2024-08-09T00:00:00.000+02:00                       59.0   
albertocanto@originaltelecom.es    2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       36.0   
                                   2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       16.0   
                                   2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       64.0   
                                   2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       42.0   
                                   2024-08-07T00:00:00.000+02:00 2024-08-08T00:00:00.000+02:00                       35.0   
                                   2024-08-08T00:00:00.000+02:00 2024-08-09T00:00:00.000+02:00                       41.0   
albertosanchez@originaltelecom.es  2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       41.0   
                                   2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       27.0   
                                   2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       45.0   
                                   2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       44.0   
                                   2024-08-07T00:00:00.000+02:00 2024-08-08T00:00:00.000+02:00                       39.0   
                                   2024-08-08T00:00:00.000+02:00 2024-08-09T00:00:00.000+02:00                       53.0   
anasanchez@originaltelecom.es      2024-08-03T00:00:00.000+02:00 2024-08-04T00:00:00.000+02:00                       55.0   
                                   2024-08-04T00:00:00.000+02:00 2024-08-05T00:00:00.000+02:00                       24.0   
                                   2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       50.0   
                                   2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       33.0   
                                   2024-08-07T00:00:00.000+02:00 2024-08-08T00:00:00.000+02:00                       45.0   
antonio.reina@originaltelecom.es   2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       30.0   
                                   2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       23.0   
                                   2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       58.0   
                                   2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       53.0   
                                   2024-08-07T00:00:00.000+02:00 2024-08-08T00:00:00.000+02:00                       33.0   
                                   2024-08-08T00:00:00.000+02:00 2024-08-09T00:00:00.000+02:00                       31.0   
beatriz.gomez@originaltelecom.es   2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       26.0   
                                   2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       28.0   
                                   2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00  

Niveles del índice: ['Agent', 'StartInterval', 'EndInterval']
Columnas del DataFrame: Index(['Contacts handled incoming', 'Contacts transferred out internal'], dtype='object')
Contenido del DataFrame:
                                                                                           Contacts handled incoming  \
Agent                         StartInterval                 EndInterval                                                
albaaraujo@originaltelecom.es 2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       33.0   
                              2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       29.0   
                              2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       63.0   
                              2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       46.0   
                              2024-08-07T00:00:00.000+02:00 2024-08-08T00:00:00

## Assign Values to Emails

Assign numerical values to each email and add them as a new column in the DataFrames.

In [6]:
# Dictionary mapping emails to their respective values
email_values = {
    'albaaraujo@originaltelecom.es': 1,
    'albertocanto@originaltelecom.es': 2,
    'albertosanchez@originaltelecom.es': 3,
    'anasanchez@originaltelecom.es': 4,
    'antonio.reina@originaltelecom.es': 5,
    'azahara.garcia@originaltelecom.es': 6,
    'beatriz.gomez@originaltelecom.es': 7,
    'carmen.cornejo@originaltelecom.es': 8,
    'carolinafuentes@originaltelecom.es': 9,
    'cesar.arnaldo@originaltelecom.es': 10,
    'david.molero@originaltelecom.es': 11,
    'dolores.cortes@originaltelecom.es': 22,
    'elenaborrero@originaltelecom.es': 12,
    'estefania.panea@originaltelecom.es': 13,
    'pilar.deval@originaltelecom.es': 39,
    'manuelventura@originaltelecom.es': 24,
    'mar.aguila@originaltelecom.es': 25,
    'mauricio.pozo@originaltelecom.es': 30,
    'francisco.perdomo@originaltelecom.es': 14,
    'gonzalofalcon@originaltelecom.es': 15,
    'guillermo.hurtado@originaltelecom.es': 16,
    'irati.izaguirre@originaltelecom.es': 17,
    'ivan.barroso@originaltelecom.es': 18,
    'lailasetati@originaltelecom.es': 20,
    'laura.eguens@originaltelecom.es': 19,
    'leonor.lopez@originaltelecom.es': 21,
    'manuelvaldes@originaltelecom.es': 23,
    'mar.marchena@originaltelecom.es': 33,
    'maria.torres@originaltelecom.es': 28,
    'mariaarroyo@originaltelecom.es': 27,
    'mariangeles.bueso@originaltelecom.es': 26,
    'marta.dorado@originaltelecom.es': 29,
    'miguel.segura@originaltelecom.es': 31,
    'miriam.rodriguez@originaltelecom.es': 32,
    'natividad.sanchez@originaltelecom.es': 34,
    'nereacerezo@originaltelecom.es': 35,
    'oscar.rivilla@originaltelecom.es': 36,
    'patricia.rios@originaltelecom.es': 37,
    'paulavilla@originaltelecom.es': 38,
    'sara.elkhelyfy@originaltelecom.es': 40,
    'sergio.vazquez@originaltelecom.es': 41,
    'yicel.patricia@originaltelecom.es': 42,
    'yzabelly.gomes@originaltelecom.es': 43
}

# Ordenar la tabla pivoteada por valores numéricos asociados a los correos electrónicos
def sort_pivot_table_by_email_values(pivot_df, email_values):
    # Crear una serie temporal con los valores de email
    temp_index = pivot_df.index.get_level_values('Agent').map(email_values)
    
    # Crear un nuevo DataFrame temporal con el índice original y los valores numéricos
    temp_df = pd.DataFrame({'temp_index': temp_index, 'original_index': pivot_df.index})
    
    # Ordenar el DataFrame temporal por los valores numéricos
    temp_df = temp_df.sort_values(by='temp_index')
    
    # Reordenar la tabla pivoteada usando el índice original ordenado
    pivot_df = pivot_df.loc[temp_df['original_index']]
    
    return pivot_df

# Aplicar la función para ordenar la tabla pivoteada
sorted_pivot_df = sort_pivot_table_by_email_values(pivot_df, email_values)

# Mostrar la tabla pivoteada ordenada
print("Informe DataFrame sorted by email values:")
print(sorted_pivot_df.head(15))

Informe DataFrame sorted by email values:
                                                                                               Contacts handled incoming  \
Agent                             StartInterval                 EndInterval                                                
albaaraujo@originaltelecom.es     2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       33.0   
                                  2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       29.0   
                                  2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       63.0   
                                  2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       46.0   
                                  2024-08-07T00:00:00.000+02:00 2024-08-08T00:00:00.000+02:00                       58.0   
                                  2024-08-08T00:00:00.000+02:00 2024-08-09T00:00:00.000+02

In [8]:

# Supongamos que pivot_df es tu DataFrame pivoteado con tres niveles de índice

# Combinar 'StartInterval' y 'EndInterval' en un solo índice de cadena
# Aquí se usa ' - ' para separar las fechas, pero puedes elegir otro separador
new_index = pivot_df.index.map(lambda x: f"{x[1]} - {x[2]}")

# Asignar el nuevo índice al DataFrame, manteniendo 'Agent' como el primer nivel de índice
pivot_df.set_index(new_index, append=True, inplace=True)

# Eliminar los índices 'StartInterval' y 'EndInterval' originales
pivot_df.reset_index(level=['StartInterval', 'EndInterval'], drop=True, inplace=True)

# Renombrar el nuevo nivel de índice para mayor claridad
pivot_df.index.names = ['Agent', 'DateInterval']

# Mostrar el DataFrame con el nuevo índice
print(pivot_df)


IndexError: tuple index out of range